In [38]:
import pystac_client
import pystac
import odc.stac
import rioxarray
import pathlib
import pandas
import numpy
import folium
import branca.element, branca.colormap # Remove whitespace around small folium map

In [39]:
def geopandas_bounds_to_plot(dataframe, crs=4326):
    """ Changing bounding box representation to leaflet notation ``(lon1, lat1, lon2, lat2) -> ((lat1, lon1), (lat2, lon2))`` """
    x1, y1, x2, y2 = dataframe.to_crs(crs).total_bounds
    return ((y1, x1), (y2, x2))

In [40]:
data_path = pathlib.Path.cwd() / ".." / "data"
(data_path / "rasters").mkdir(parents=True, exist_ok=True)
crs_wsg = 4326
crs = 2193
name = "waikouaiti"

In [41]:
bands = ["red", "green", "blue", "nir", "scl"]
SCL_CIRRUS = 10
SCL_CLOUD = 9
SCL_DEFECTIVE = 1
SCL_NO_DATA = 0
offset = -0.1
scale = 0.0001
thresholds = {"min_ndvi": 0.01, "max_ndvi": 0.7, "max_ndwi": 0.2}

In [42]:
earth_search = {"url": "https://earth-search.aws.element84.com/v1", "collection": "sentinel-2-c1-l2a"}
microsoft = {"url": "https://planetarycomputer.microsoft.com/api/stac/v1", "collection": "sentinel-2-l2a"}

stec = earth_search

In [43]:
# use publically available stac link such as
odc.stac.configure_rio(cloud_defaults=True, aws={"aws_unsigned": True})
client = pystac_client.Client.open(stec["url"]) 

# ID of the collection
collection = stec["collection"]

# Geometry of AOI
import geopandas
geometry_df = geopandas.read_file(data_path / "vectors" / f"{name}.gpkg")
geometry = geometry_df.to_crs(crs_wsg).iloc[0].geometry
land = geopandas.read_file(data_path / "vectors" / f"main_islands.gpkg")

In [44]:
# Complete month
date_YYMM = "2020-05-31"
filters = {"eo:cloud_cover":{"lt":10}} 
# run pystac client search to see available dataset
search = client.search(
    collections=[collection], intersects=geometry, datetime=date_YYMM, query=filters
) 

results_dict = search.item_collection_as_dict()
pandas.DataFrame.from_records(results_dict['features'])

,type,stac_version,id,properties,geometry,links,assets,bbox,stac_extensions,collection
0,Feature,1.0.0,S2A_T59GMK_20200531T223733_L2A,"{'created': '2024-01-22T04:29:28.342Z', 'platf...","{'type': 'Polygon', 'coordinates': [[[169.7274...","[{'rel': 'self', 'type': 'application/geo+json...",{'red': {'href': 'https://e84-earth-search-sen...,"[169.704927, -46.140291, 171.115971, -45.146216]",[https://stac-extensions.github.io/eo/v1.1.0/s...,sentinel-2-c1-l2a


## Check expected offsets and scales
Do this before downloading with groupby solar day

In [45]:
for item in search.item_collection():
    for band in bands:
        if band == "scl": 
            continue
        if item.assets[band].extra_fields['raster:bands'][0]['offset'] != offset:
            raise Exception("offset doesn't match expected")
        if item.assets[band].extra_fields['raster:bands'][0]['scale'] != scale:
            raise Exception("scale doesn't match expected")

## Download and constuct Kelp layer

In [46]:
data = odc.stac.load(search.items(), geopolygon=geometry, bands=bands,  chunks={}, groupby="solar_day")

### Remove any dates with no valid data

In [47]:
data["scl"].load()
data["scl"] = data["scl"].rio.clip(land.to_crs(data["scl"].rio.crs).geometry.values, invert=True)
data["scl"].rio.write_crs(data["scl"].rio.crs, inplace=True);
data.isel(time=(data["scl"] != SCL_NO_DATA).any(dim=["x", "y"]));

## Caclulate kelp for remaining dates

In [48]:
for band in bands: 
    if band == "scl": 
        continue
    data[band].data = data[band].data * scale + offset
    data[band].data[data[band].data <= 0] = numpy.nan
    data[band].rio.write_nodata(numpy.nan, encoded=True, inplace=True);

In [49]:
data["ndvi"] = (data.nir - data.red) / (data.nir + data.red)
data["ndwi"] = (data.green-data.nir)/(data.green+data.nir)
data["ndvi"].rio.write_crs(data["ndvi"].rio.crs, inplace=True); data["ndvi"].rio.write_nodata(numpy.nan, encoded=True, inplace=True);
data["ndwi"].rio.write_crs(data["ndvi"].rio.crs, inplace=True); data["ndwi"].rio.write_nodata(numpy.nan, encoded=True, inplace=True); 

In [50]:
land = geopandas.read_file(data_path / "vectors" / "main_islands.geojson")

In [51]:
data["kelp"] = (data.nir - data.red) / (data.nir + data.red)
data["kelp"].data[(data["ndvi"].data < thresholds["min_ndvi"]) | (data["ndvi"].data > thresholds["max_ndvi"]) | (data["ndwi"].data > thresholds["max_ndwi"])] = numpy.nan
data["kelp"].data[(data["scl"].data == SCL_CIRRUS) | (data["ndvi"].data == SCL_CLOUD) | (data["ndwi"].data == SCL_DEFECTIVE)] = numpy.nan
data["kelp"] = data["kelp"].rio.clip(land.to_crs(data["kelp"].rio.crs).geometry.values, invert=True)
data["kelp"].rio.write_crs(data["kelp"].rio.crs, inplace=True); data["kelp"].rio.write_nodata(numpy.nan, encoded=True, inplace=True);

In [52]:
data["kelp"].to_netcdf(data_path / "rasters" / f'kelp_month_{name}_1.nc', format="NETCDF4", engine="netcdf4")
data.to_netcdf(data_path / "rasters" / f'all_month_{name}.nc', format="NETCDF4", engine="netcdf4")

In [ ]:
kelp_display = data["kelp"].isel(time=0)

fig = branca.element.Figure(width='70%', height='50%') # Ensures no extra whitespace below

m = folium.Map()
land.explore(m=m)
kelp_display.odc.add_to(m, opacity=0.75, cmap="inferno", vmin=0, vmax=1) # viridis
m.fit_bounds(kelp_display.odc.map_bounds())

colormap = branca.colormap.linear.inferno.scale(0, 1)
colormap.caption = 'Kelp Index'
colormap.add_to(m)

display(m)